In [56]:
#Connect to GGE API

import ee
import geemap.core as geemap
import datetime

ee.Initialize(project='ee-sand-engine')

Selected collection: COPERNICUS/S2_SR
Analysis: NDWI
Bands to use: Green = ['B3', 'B8']
Number of images: 203
Average time interval between images: 13.153465346534654 days
Exporting myExportVideoTask... Please check the Task Manager from the JavaScript Code Editor.


In [51]:
#Connect to GGE API

import ee
import geemap.core as geemap
import datetime

ee.Initialize(project='ee-sand-engine')

# Define variables

start_date = '2011-01-01'
end_date = '2024-12-31'

geometry = ee.Geometry.Polygon([[
    [4.161971, 52.026261],
    [4.229327, 52.075821],
    [4.198982, 52.091474],
    [4.131249, 52.041400],
    [4.131249, 52.041400]
]])


"""   
# NDWI Analysis
COPERNICUS/S2 or COPERNICUS/S2_SR (Sentinel 2)    Green: B3,  NIR: B8
LANDSAT/LC09/C02/T1_L2 (Landsat 9)                Green: B3,  NIR: B5
LANDSAT/LC08/C02/T1_L2 (Landsat 8)                Green: B3,  NIR: B5
LANDSAT/LE07/C02/T1_L2 (Landsat 7)                Green: B2,  NIR: B4
LANDSAT/LT05/C02/T1_L2 (Landsat 5)                Green: B2,  NIR: B4
# MNDWI Analysis
COPERNICUS/S2 or COPERNICUS/S2_SR (Sentinel 2):   Green: B3,  SWIR: B11
LANDSAT/LC09/C02/T1_L2 (Landsat 9):               Green: B3,  SWIR: B6
LANDSAT/LC08/C02/T1_L2 (Landsat 8):               Green: B3,  SWIR: B6
LANDSAT/LE07/C02/T1_L2 (Landsat 7):               Green: B2,  SWIR: B5
LANDSAT/LT05/C02/T1_L2 (Landsat 5):               Green: B2,  SWIR: B5
"""

# Dictionary of collections and corresponding bands
band_info = {
    "COPERNICUS/S2_SR": {
        "NDWI": ("B3", "B8"),    # Green, NIR
        "MNDWI": ("B3", "B11")  # Green, SWIR
    },
    "COPERNICUS/S2": {
        "NDWI": ("B3", "B8"),
        "MNDWI": ("B3", "B11")
    },
    "LANDSAT/LC09/C02/T1_L2": {
        "NDWI": ("B3", "B5"),
        "MNDWI": ("B3", "B6")
    },
    "LANDSAT/LC08/C02/T1_L2": {
        "NDWI": ("B3", "B5"),
        "MNDWI": ("B3", "B6")
    },
    "LANDSAT/LE07/C02/T1_L2": {
        "NDWI": ("B2", "B4"),
        "MNDWI": ("B2", "B5")
    },
    "LANDSAT/LT05/C02/T1_L2": {
        "NDWI": ("B2", "B4"),
        "MNDWI": ("B2", "B5")
    }
}

collection_name = "COPERNICUS/S2_SR"  
analysis_type = "NDWI"              
green_band, other_band = band_info[collection_name][analysis_type]
bands = [green_band, other_band]

print(f"Selected collection: {collection_name}")
print(f"Analysis: {analysis_type}")
print(f"Bands to use: Green = {bands}")

# Define cloud cover filter
cloud_cover_max = 30  # Maximum cloud cover percentage

# Filter the image collection
collection = (ee.ImageCollection(collection_name)
              .filterDate(start_date, end_date)
              .filterBounds(geometry)
              .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', cloud_cover_max))

# Get statistics
image_count = collection.size().getInfo()
dates = collection.aggregate_array('system:time_start').getInfo()
dates = [datetime.datetime.utcfromtimestamp(d / 1000) for d in dates]
time_intervals = [(dates[i] - dates[i-1]).days for i in range(1, len(dates))]
average_interval = sum(time_intervals) / len(time_intervals) if time_intervals else 0

# Print statistics
print(f'Number of images: {image_count}')
print(f'Average time interval between images: {average_interval} days')

# Calculate NDWI
def calculate_ndwi(image):
    ndwi = image.normalizedDifference(bands)
    return ndwi.rename('NDWI')

# Map the function over the collection
ndwi_collection = collection.map(calculate_ndwi)

# Create a map of the NDWI
Map = geemap.Map()
# Display NDWI of the first image
first_ndwi = ndwi_collection.first().clip(geometry)
Map.addLayer(
    first_ndwi, 
    {"min": -0.1, "max": 0.7, "palette": ["green", "white", "blue"]}, 
    "NDWI (First Image)"
)
Map.centerObject(geometry)
Map

Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…

In [13]:
import os
import requests
import json
import ee

ee.Initialize(project='ee-sand-engine')
os.environ['PL_API_KEY']='PLAK5b25ad60f8c14eb381e4c2511439d0e2'
# pass in your API key

PLANET_API_KEY = os.getenv('PL_API_KEY')
# Setup the API Key from the `PL_API_KEY` environment variable

BASE_URL = "https://api.planet.com/data/v1"

# Sample Geometry (GeoJSON) for bounding box
geometry_filter = {
   "type": "GeometryFilter",
   "field_name": "geometry",
   "config": {
       "type": "Polygon",
       "coordinates": [[
            [4.161971, 52.026261],
            [4.229327, 52.075821],
            [4.198982, 52.091474],
            [4.131249, 52.041400],
            [4.131249, 52.041400]
       ]]
   }
}

# Create a simple filter for our item types
item_types = ["PSScene4Band"]  # PlanetScope 4-band as an example

search_endpoint = f"{BASE_URL}/searches"

search_payload = {
   "item_types": item_types,
   "filter": geometry_filter
}

# Create a session
session = requests.Session()
session.auth = (PLANET_API_KEY, "")

# POST the search
search_res = session.post(search_endpoint, json=search_payload)
print("Search response status:", search_res.status_code)
if search_res.status_code == 200:
    search_results = search_res.json()
    print(json.dumps(search_results, indent=2))
else:
    print("Error searching Planet:", search_res.text)

Search response status: 400
Error searching Planet: {"field": {"name": [{"message": "'name' is a required property"}]}, "general": []}


In [23]:
import geemap
import os
import base64
import requests
import json
import ee

ee.Initialize(project='ee-sand-engine')

# 1) Your Planet API key
PLANET_API_KEY = "PLAK5b25ad60f8c14eb381e4c2511439d0e2"
BASE_URL = "https://api.planet.com/data/v1"

# 2) Set up Basic Auth headers
encoded_credentials = base64.b64encode(f"{PLANET_API_KEY}:".encode("utf-8")).decode("utf-8")
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Basic {encoded_credentials}"
}

# 3) Define your geometry (GeoJSON) and filters

with open("D:/My Documents/Thesis/Coastal_vision/explorer-aoi.geojson", "r") as f:
    AOI = json.load(f)

if AOI["type"] == "FeatureCollection":
    AOI = AOI["features"][0]["geometry"]  # Use the first feature's geometry
else:
    AOI = AOI
# # AOI = {
#     "type": "Polygon",
#     "coordinates": [[
#         [4.161971, 52.026261],
#         [4.229327, 52.075821],
#         [4.198982, 52.091474],
#         [4.131249, 52.041400],
#         [4.131249, 52.041400]
#     ]]
# }


START_DATE = '2015-12-01T00:00:00Z'
END_DATE   = '2024-12-31T23:59:59Z'
CLOUD_COVER = 1  # maximum 10% cloud cover

# If you want 3-, 4-, or 8-band PlanetScope imagery, list the corresponding item types.
# Typically: "PSScene3Band", "PSScene4Band". (8-band may require a different item type)
item_types = ["PSScene"]

search_filter = {
    "type": "AndFilter",
    "config": [
        {
            "type": "GeometryFilter",
            "field_name": "geometry",
            "config": AOI
        },
        {
            "type": "DateRangeFilter",
            "field_name": "acquired",
            "config": {"gte": START_DATE, "lte": END_DATE}
        },
        {
            "type": "RangeFilter",
            "field_name": "cloud_cover",
            "config": {"lte": CLOUD_COVER}
        # },
        # # Filter by instrument name for PS2, PS2.SD, or PSB.SD:
        # {
        #     "type": "StringInFilter",
        #     "field_name": "instrument",
        #     "config": ["PS2", "PS2.SD", "PSB.SD"]
         }
    ]
}

payload = {
    "name": "PlanetScope_Sensors_Search",
    "item_types": item_types,
    "filter": search_filter
}

# 4) POST the search
search_url = f"{BASE_URL}/searches"
response = requests.post(search_url, headers=headers, json=payload)

if response.status_code == 200:
    data = response.json()
    print(json.dumps(data, indent=2))  # Print the entire response for inspection

    features = data.get("features", [])
    print(f"Found {len(features)} matching images for these instruments.")
    
    # For each feature, activate the desired asset
    for feat in features:
        item_id = feat["id"]
        assets_link = feat["_links"]["assets"]
        # Get the available assets
        assets_res = requests.get(assets_link, headers=headers)
        if assets_res.status_code == 200:
            assets = assets_res.json()
            # For example, check if "analytic_sr" is available
            if "analytic_sr" in assets:
                activate_url = assets["analytic_sr"]["_links"]["activate"]
                act_res = requests.post(activate_url, headers=headers)
                if act_res.status_code in [204, 202]:
                    print(f"Activated asset for item {item_id} (PSScene).")
                else:
                    print(f"Failed to activate asset for item {item_id}.")
            else:
                print(f"No analytic_sr asset found for item {item_id}.")
        else:
            print(f"Error retrieving assets for item {item_id}: {assets_res.text}")
else:
    print(f"Search failed with status {response.status_code}: {response.text}")

{
  "__daily_email_enabled": false,
  "_links": {
    "_self": "https://api.planet.com/data/v1/searches/20adc7c04c4f46a991af5ff3514d603d",
    "results": "https://api.planet.com/data/v1/searches/20adc7c04c4f46a991af5ff3514d603d/results"
  },
  "created": "2025-01-07T17:36:45.155473Z",
  "filter": {
    "config": [
      {
        "config": {
          "coordinates": [
            [
              [
                4.14933522,
                52.03779721
              ],
              [
                4.20370822,
                52.07721943
              ],
              [
                4.22296841,
                52.06966544
              ],
              [
                4.16726202,
                52.02987215
              ],
              [
                4.14933522,
                52.03779721
              ]
            ]
          ],
          "type": "Polygon"
        },
        "field_name": "geometry",
        "type": "GeometryFilter"
      },
      {
        "config": {
 

In [26]:
import getpass
from planet import Auth

user = input("Username: ")
pw = getpass.getpass()
auth = Auth.from_login(user,pw)
auth.store()

ModuleNotFoundError: No module named 'planet'

In [1]:
import asyncio
from planet import Auth, Session, DataClient
from planet.order_request import build_request

async def main():
    # Define Area of Interest (AOI)
    aoi = {
        "type": "Polygon",
        "coordinates": [[
            [-122.5190, 37.7045],
            [-122.5190, 37.8520],
            [-122.3550, 37.8520],
            [-122.3550, 37.7045],
            [-122.5190, 37.7045]
        ]]
    }

    # Authentication
    API_KEY = "PLAK5b25ad60f8c14eb381e4c2511439d0e2"
    auth = Auth.from_key(API_KEY)

    # Set up a session
    async with Session(auth=auth) as session:
        client = DataClient(session)

        # Define search filter
        search_filter = {
            "type": "AndFilter",
            "config": [
                {"type": "GeometryFilter", "field_name": "geometry", "config": aoi},
                {"type": "DateRangeFilter", "field_name": "acquired", "config": {"gte": "2021-01-01T00:00:00Z", "lte": "2021-12-31T23:59:59Z"}},
                {"type": "RangeFilter", "field_name": "cloud_cover", "config": {"lte": 0.5}},
                {"type": "StringInFilter", "field_name": "instrument", "config": ["PS2", "PS2.SD", "PSB.SD"]}
            ]
        }

        # Perform the search
        request = await client.search(
            name="PlanetScope_Sensors_Search",
            item_types=["PSScene"],
            filter=search_filter
        )

        # Collect results
        results = [r async for r in request.items_iter()]
        print(f"Found {len(results)} matching images for these instruments.")

        # Activate assets
        for item in results:
            item_id = item["id"]
            assets = await client.get_assets(item_id)

            if 'analytic_sr' in assets:
                activation = await client.activate_asset(assets['analytic_sr'])
                if activation.status in ["active", "activating"]:
                    print(f"Activated asset for item {item_id} (PSScene).")
                else:
                    print(f"Failed to activate asset for item {item_id}.")
            else:
                print(f"No analytic_sr asset found for item {item_id}.")

# Run the main function
asyncio.run(main())

ModuleNotFoundError: No module named 'planet'

In [53]:
#Clear cache
import os
import ee
import geemap
import datetime

def initialize_gee(project_name):
    """Initialize GEE with specific project"""
    ee.Initialize(project=project_name)

def get_user_input():
    """Get processing parameters from user"""
    print("\nAvailable Collections:")
    collections = [
        "COPERNICUS/S2_SR",
        "COPERNICUS/S2",
        "LANDSAT/LC09/C02/T1_L2",
        "LANDSAT/LC08/C02/T1_L2",
        "LANDSAT/LE07/C02/T1_L2",
        "LANDSAT/LT05/C02/T1_L2"
    ]
    
    for i, collection in enumerate(collections, 1):
        print(f"{i}. {collection}")
    
    collection_idx = int(input("\nSelect collection number: ")) - 1
    collection_name = collections[collection_idx]
    
    print("\nAvailable Indices:")
    print("1. NDWI")
    print("2. MNDWI")
    
    index_choice = input("\nSelect index (1 or 2): ")
    index_type = "NDWI" if index_choice == "1" else "MNDWI"
    
    start_date = input("\nEnter start date (YYYY-MM-DD): ")
    end_date = input("Enter end date (YYYY-MM-DD): ")
    cloud_cover = float(input("Enter maximum cloud cover percentage (0-100): "))
    
    return collection_name, index_type, start_date, end_date, cloud_cover

def mask_clouds_landsat(image):
    """Mask clouds and cloud shadows in Landsat imagery"""
    qa = image.select('QA_PIXEL')
    cloudShadowBitMask = 1 << 4
    cloudsBitMask = 1 << 3
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
        .And(qa.bitwiseAnd(cloudsBitMask).eq(0))
    return image.updateMask(mask)

def mask_clouds_sentinel2(image):
    """Mask clouds and cloud shadows in Sentinel-2 imagery"""
    qa = image.select('QA60')
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
        .And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask)

def apply_scale_factors(image, collection_name):
    """Apply appropriate scale factors based on collection"""
    if 'LANDSAT' in collection_name:
        optical_bands = image.select(['SR_B.|SR_B2|SR_B3|SR_B4|SR_B5|SR_B6|SR_B7'])
        scaled = optical_bands.multiply(0.0000275).add(-0.2)
    else:  # Sentinel-2
        optical_bands = image.select(['B.|B2|B3|B4|B5|B6|B7|B8|B8A|B11|B12'])
        scaled = optical_bands.multiply(0.0001)
    
    return image.addBands(scaled, None, True)

def calculate_water_index(image, green_band, other_band, index_type):
    """Calculate water index (NDWI or MNDWI)"""
    water_index = image.normalizedDifference([green_band, other_band]).rename(index_type)
    return image.addBands(water_index)

def process_collection(collection_name, index_type, start_date, end_date, geometry, cloud_cover_max):
    """Process image collection with appropriate masks and scaling"""
    # Get band information
    band_info = {
        "COPERNICUS/S2_SR": {
            "NDWI": ("B3", "B8"),
            "MNDWI": ("B3", "B11")
        },
        "COPERNICUS/S2": {
            "NDWI": ("B3", "B8"),
            "MNDWI": ("B3", "B11")
        },
        "LANDSAT/LC09/C02/T1_L2": {
            "NDWI": ("SR_B3", "SR_B5"),
            "MNDWI": ("SR_B3", "SR_B6")
        },
        "LANDSAT/LC08/C02/T1_L2": {
            "NDWI": ("SR_B3", "SR_B5"),
            "MNDWI": ("SR_B3", "SR_B6")
        },
        "LANDSAT/LE07/C02/T1_L2": {
            "NDWI": ("SR_B2", "SR_B4"),
            "MNDWI": ("SR_B2", "SR_B5")
        },
        "LANDSAT/LT05/C02/T1_L2": {
            "NDWI": ("SR_B2", "SR_B4"),
            "MNDWI": ("SR_B2", "SR_B5")
        }
    }
    
    green_band, other_band = band_info[collection_name][index_type]
    print(f"\nUsing bands: Green = {green_band}, Other = {other_band}")
    
    # Initialize collection
    collection = ee.ImageCollection(collection_name) \
        .filterDate(start_date, end_date) \
        .filterBounds(geometry)
    
    if 'LANDSAT' in collection_name:
        collection = collection.filter(ee.Filter.lt('CLOUD_COVER', cloud_cover_max))
    else:  # Sentinel-2
        collection = collection.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_cover_max))
    
    def process_image(image):
    #     """Process individual image with cloud mask, scaling, and index calculation"""
    #     # Apply cloud mask
    #     if 'LANDSAT' in collection_name:
    #         image = mask_clouds_landsat(image)
    #     else:
    #         image = mask_clouds_sentinel2(image)
        
    #     # Apply scaling
    #     image = apply_scale_factors(image, collection_name)
        
    #     # Calculate water index
         image = calculate_water_index(image, green_band, other_band, index_type)
        
         return image
    
     # Apply processing to collection
    processed = collection.map(process_image)
    
    return processed

def main():
    # Define study area (Sand Engine coordinates)
    geometry = ee.Geometry.Polygon([[
        [4.161971, 52.026261],
        [4.229327, 52.075821],
        [4.198982, 52.091474],
        [4.131249, 52.041400],
        [4.131249, 52.041400]
    ]])
    
    # Initialize GEE
    initialize_gee('ee-sand-engine')
    
    # Get user inputs
    collection_name, index_type, start_date, end_date, cloud_cover = get_user_input()
    
    # Process collection
    collection = process_collection(
        collection_name, 
        index_type, 
        start_date, 
        end_date, 
        geometry, 
        cloud_cover
    )
    
    # Get collection statistics
    image_count = collection.size().getInfo()
    dates = collection.aggregate_array('system:time_start').getInfo()
    dates = [datetime.datetime.utcfromtimestamp(d / 1000) for d in dates]
    time_intervals = [(dates[i] - dates[i-1]).days for i in range(1, len(dates))]
    average_interval = sum(time_intervals) / len(time_intervals) if time_intervals else 0
    
    # Print statistics
    print(f'\nProcessing Results:')
    print(f'Number of images: {image_count}')
    print(f'Average time interval between images: {average_interval:.1f} days')
    
    # # Create visualization
    # Map = geemap.Map()
    # first_image = ee.Image(collection.first()).clip(geometry)
    # Map.addLayer(first_image, {'bands': bands, 'min': 0, 'max': 3000}, 'First Image')
    # Map.centerObject(geometry)
    # Map

    print('estas aqui')
    
    # if image_count > 0:
    #     # Calculate median composite
    #     median_composite = collection.select(index_type).median()
        
    #     # Add layers to map
    #     # First, add the raw composite
    #     viz_params_raw = {
    #         'min': -1,
    #         'max': 1,
    #         'palette': ['red', 'white', 'blue']
    #     }
    #     Map.addLayer(median_composite.clip(geometry), viz_params_raw, f'Median {index_type}')
        
    #     # Then, add a thresholded version to highlight water
    #     water_threshold = 0  # Adjust this threshold as needed
    #     water_mask = median_composite.gt(water_threshold)
    #     viz_params_water = {
    #         'min': 0,
    #         'max': 1,
    #         'palette': ['white', 'blue']
    #     }
    #     Map.addLayer(water_mask.clip(geometry), viz_params_water, f'{index_type} Water Mask')
        
    #     Map.centerObject(geometry, 12)
    # else:
    #     print("No images found matching the criteria!")
    
    return collection

if __name__ == "__main__":
    collection2 = main()
    # Map = geemap.Map()
    # first_image = ee.Image(collection.first()).clip(geometry)
    # Map.addLayer(first_image, {'bands': bands, 'min': 0, 'max': 3000}, 'First Image')
    # Map.centerObject(geometry)
    # Map

Exporting myExportVideoTask... Please check the Task Manager from the JavaScript Code Editor.


In [44]:
Map = geemap.Map()
first_image = ee.Image(collection.first()).clip(geometry)
Map.addLayer(first_image, {'bands': bands, 'min': 0, 'max': 3000}, 'First Image')
Map.centerObject(geometry)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…